In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('AsuransiProperti.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256136 entries, 0 to 256135
Data columns (total 42 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256136 non-null  int64  
 1   POLICY_NUM             256136 non-null  object 
 2   QUOTE_DATE             129524 non-null  object 
 3   COVER_START            189021 non-null  object 
 4   P1_EMP_STATUS          189021 non-null  object 
 5   BUS_USE                189021 non-null  object 
 6   AD_BUILDINGS           189021 non-null  object 
 7   RISK_RATED_AREA_B      140876 non-null  float64
 8   SUM_INSURED_BUILDINGS  189021 non-null  float64
 9   NCD_GRANTED_YEARS_B    189021 non-null  float64
 10  AD_CONTENTS            189021 non-null  object 
 11  RISK_RATED_AREA_C      180290 non-null  float64
 12  SUM_INSURED_CONTENTS   189021 non-null  float64
 13  NCD_GRANTED_YEARS_C    189021 non-null  float64
 14  CONTENTS_COVER         189021 non-nu

In [4]:
df

,ID,POLICY_NUM,QUOTE_DATE,COVER_START,P1_EMP_STATUS,BUS_USE,AD_BUILDINGS,RISK_RATED_AREA_B,SUM_INSURED_BUILDINGS,NCD_GRANTED_YEARS_B,...,OWNERSHIP_TYPE,PAYING_GUESTS,PROP_TYPE,SAFE_INSTALLED,SEC_DISC_REQ,SUBSIDENCE,YEARBUILT,PAYMENT_METHOD,PAYMENT_FREQUENCY,ANNUAL_PREMIUM
0,1,P000001,11/22/2007,22-11-2007,R,N,Y,19.0,1000000.0,7.0,...,8.0,0.0,10.0,Y,Y,N,1960.0,PureDD,NaN,274.81
1,2,P000002,11/22/2007,01-01-2008,E,Y,Y,25.0,1000000.0,6.0,...,3.0,0.0,2.0,N,N,N,1960.0,PureDD,NaN,308.83
2,3,P000003,11/23/2007,23-11-2007,E,N,N,NaN,0.0,0.0,...,8.0,0.0,9.0,N,Y,N,1946.0,PureDD,NaN,52.65
3,4,P000004,11/23/2007,12-12-2007,R,N,N,NaN,0.0,0.0,...,18.0,0.0,19.0,N,Y,N,1870.0,NonDD,NaN,54.23
4,5,P000005,11/22/2007,15-12-2007,R,N,Y,5.0,1000000.0,7.0,...,8.0,0.0,1.0,N,Y,N,1960.0,DD-Other,NaN,244.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256131,256132,P256132,NaN,22-02-2005,R,N,Y,16.0,1000000.0,2.0,...,8.0,0.0,2.0,N,N,N,1980.0,PureDD,1.0,235.08
256132,256133,P256133,NaN,01-12-2000,R,N,Y,0.0,1000000.0,5.0,...,8.0,0.0,1.0,N,Y,N,1980.0,NonDD,1.0,194.02
256133,256134,P256134,NaN,18-01-2006,R,N,Y,1.0,1000000.0,5.0,...,8.0,0.0,19.0,N,Y,N,1900.0,PureDD,1.0,287.30
256134,256135,P256135,NaN,31-12-2004,R,N,Y,32.0,1000000.0,5.0,...,3.0,0.0,19.0,N,Y,N,1900.0,DD-Other,1.0,457.57


In [5]:
cols_to_drop = [
    'ID',                   # identifier unik, tidak perlu sebagai fitur
    'POLICY_NUM',           # identifier lain
    'QUOTE_DATE',           # rencana olah jadi fitur baru, tapi sementara hapus dulu
    'COVER_START',          # rencana olah jadi fitur baru, tapi sementara hapus dulu
    'P1_EMP_STATUS',        # evaluasi pengaruh, sementara hapus dulu
    'SUM_INSURED_BUILDINGS',# fitur dengan banyak missing / unik, hapus dulu
    'NCD_GRANTED_YEARS_B',  # variabel redundant, hapus
    'PAYING_GUESTS',        # hapus
    'PAYMENT_METHOD',       # hapus
    'PAYMENT_FREQUENCY',    # banyak missing, hapus
    'RISK_RATED_AREA_B',
    'RISK_RATED_AREA_C',
    'P1_POLICY_REFUSED',
    'SAFE_INSTALLED',
    'P1_DOB',
    'SPEC_SUM_INSURED',
    'ROOF_CONSTRUCTION',
    'WALL_CONSTRUCTION',
    'BUILDINGS_COVER',
    'AD_CONTENTS',
    'SUM_INSURED_CONTENTS',
    'NCD_GRANTED_YEARS_C',
    'SEC_DISC_REQ',
    'MAX_DAYS_UNOCC',
    'LISTED',
    'BEDROOMS',
    'APPR_LOCKS',
    'P1_GENDER',
    'P1_MAR_STATUS',
    'SPEC_ITEM_PREM',
    'AD_BUILDINGS',
    'BUS_USE',
    'OCC_STATUS'
]

In [6]:
df = df.drop(columns=cols_to_drop, errors='ignore')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256136 entries, 0 to 256135
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CONTENTS_COVER  189021 non-null  object 
 1   APPR_ALARM      189021 non-null  object 
 2   FLOODING        189021 non-null  object 
 3   NEIGH_WATCH     189021 non-null  object 
 4   OWNERSHIP_TYPE  189021 non-null  float64
 5   PROP_TYPE       189021 non-null  float64
 6   SUBSIDENCE      189021 non-null  object 
 7   YEARBUILT       189021 non-null  float64
 8   ANNUAL_PREMIUM  190039 non-null  float64
dtypes: float64(4), object(5)
memory usage: 17.6+ MB


In [7]:
df.rename(columns={
    "CONTENTS_COVER": "Perlindungan Isi Rumah",
    "APPR_ALARM": "Alarm Terpasang",
    "FLOODING": "Risiko Banjir",
    "NEIGH_WATCH": "Keanggotaan Keamanan Lingkungan",
    "OWNERSHIP_TYPE": "Jenis Kepemilikan Properti",
    "PROP_TYPE": "Tipe Properti",
    "SUBSIDENCE": "Risiko Penurunan Tanah",
    "YEARBUILT": "Tahun Dibangun",
    "ANNUAL_PREMIUM": "Premi Asuransi"
}, inplace=True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256136 entries, 0 to 256135
Data columns (total 9 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Perlindungan Isi Rumah           189021 non-null  object 
 1   Alarm Terpasang                  189021 non-null  object 
 2   Risiko Banjir                    189021 non-null  object 
 3   Keanggotaan Keamanan Lingkungan  189021 non-null  object 
 4   Jenis Kepemilikan Properti       189021 non-null  float64
 5   Tipe Properti                    189021 non-null  float64
 6   Risiko Penurunan Tanah           189021 non-null  object 
 7   Tahun Dibangun                   189021 non-null  float64
 8   Premi Asuransi                   190039 non-null  float64
dtypes: float64(4), object(5)
memory usage: 17.6+ MB


,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi
0,Y,N,Y,N,8.0,10.0,N,1960.0,274.81
1,Y,N,Y,N,3.0,2.0,N,1960.0,308.83
2,N,Y,Y,Y,8.0,9.0,N,1946.0,52.65
3,N,N,Y,N,18.0,19.0,N,1870.0,54.23
4,Y,Y,Y,N,8.0,1.0,N,1960.0,244.58
...,...,...,...,...,...,...,...,...,...
256131,Y,N,Y,N,8.0,2.0,N,1980.0,235.08
256132,Y,N,Y,N,8.0,1.0,N,1980.0,194.02
256133,Y,Y,Y,N,8.0,19.0,N,1900.0,287.30
256134,Y,N,Y,N,3.0,19.0,N,1900.0,457.57


In [8]:
# Untuk kolom 'Jenis Kepemilikan Properti' (OWNERSHIP_TYPE)
print(df['Jenis Kepemilikan Properti'].value_counts())

# Untuk kolom 'Tipe Properti' (PROP_TYPE)
print(df['Tipe Properti'].value_counts())

Jenis Kepemilikan Properti
8.0     149755
3.0      27388
12.0      4688
18.0      3647
14.0      2610
2.0        416
7.0        158
11.0       118
13.0       109
17.0        83
16.0        28
6.0         12
1.0          5
15.0         4
Name: count, dtype: int64
Tipe Properti
10.0    55230
2.0     32089
1.0     29776
19.0    28948
9.0     16224
18.0     6362
25.0     5924
7.0      5767
26.0     2535
48.0     1803
51.0      678
4.0       631
17.0      625
32.0      557
16.0      359
53.0      320
45.0      315
47.0      312
52.0      114
31.0       93
22.0       66
40.0       51
29.0       28
27.0       27
20.0       27
30.0       25
3.0        23
37.0       23
44.0       21
21.0       20
39.0       16
24.0       14
38.0        8
28.0        5
23.0        4
8.0         1
Name: count, dtype: int64


Jenis Kepemilikan Properti 

    1: "Lainnya",
    2: "Hak Milik",
    3: "Sewa",
    4: "Warisan / Hibah",
    5: "Rumah Dinas",
    6: "KPR",
    7: "Kontrak",
    8: "HGB",
    9: "Sewa Beli",
    10: "Lainnya"

Tipe Properti

    1: "Rumah Tunggal",
    2: "Apartemen",
    3: "Rumah Susun",
    4: "Rumah Terpisah",
    5: "Ruko",
    6: "Vila",
    7: "Kios",
    8: "Petak",
    9: "Studio",
    10: "Kontrakan",

In [9]:
import pandas as pd

# Hitung distribusi dan urutkan
original_counts = df['Jenis Kepemilikan Properti'].value_counts()
sorted_categories = original_counts.index.tolist()
new_ownership_mapping = {cat: i+1 for i, cat in enumerate(sorted_categories)}

# Timpa kolom aslinya dengan kode baru
df['Jenis Kepemilikan Properti'] = df['Jenis Kepemilikan Properti'].map(new_ownership_mapping)
df

,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi
0,Y,N,Y,N,1.0,10.0,N,1960.0,274.81
1,Y,N,Y,N,2.0,2.0,N,1960.0,308.83
2,N,Y,Y,Y,1.0,9.0,N,1946.0,52.65
3,N,N,Y,N,4.0,19.0,N,1870.0,54.23
4,Y,Y,Y,N,1.0,1.0,N,1960.0,244.58
...,...,...,...,...,...,...,...,...,...
256131,Y,N,Y,N,1.0,2.0,N,1980.0,235.08
256132,Y,N,Y,N,1.0,1.0,N,1980.0,194.02
256133,Y,Y,Y,N,1.0,19.0,N,1900.0,287.30
256134,Y,N,Y,N,2.0,19.0,N,1900.0,457.57


In [10]:
original_counts_prop = df['Tipe Properti'].value_counts()
sorted_categories_prop = original_counts_prop.index.tolist()
new_prop_mapping = {cat: i+1 for i, cat in enumerate(sorted_categories_prop)}

# Ganti langsung kolom aslinya
df['Tipe Properti'] = df['Tipe Properti'].map(new_prop_mapping)
df

,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi
0,Y,N,Y,N,1.0,1.0,N,1960.0,274.81
1,Y,N,Y,N,2.0,2.0,N,1960.0,308.83
2,N,Y,Y,Y,1.0,5.0,N,1946.0,52.65
3,N,N,Y,N,4.0,4.0,N,1870.0,54.23
4,Y,Y,Y,N,1.0,3.0,N,1960.0,244.58
...,...,...,...,...,...,...,...,...,...
256131,Y,N,Y,N,1.0,2.0,N,1980.0,235.08
256132,Y,N,Y,N,1.0,3.0,N,1980.0,194.02
256133,Y,Y,Y,N,1.0,4.0,N,1900.0,287.30
256134,Y,N,Y,N,2.0,4.0,N,1900.0,457.57


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256136 entries, 0 to 256135
Data columns (total 9 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Perlindungan Isi Rumah           189021 non-null  object 
 1   Alarm Terpasang                  189021 non-null  object 
 2   Risiko Banjir                    189021 non-null  object 
 3   Keanggotaan Keamanan Lingkungan  189021 non-null  object 
 4   Jenis Kepemilikan Properti       189021 non-null  float64
 5   Tipe Properti                    189021 non-null  float64
 6   Risiko Penurunan Tanah           189021 non-null  object 
 7   Tahun Dibangun                   189021 non-null  float64
 8   Premi Asuransi                   190039 non-null  float64
dtypes: float64(4), object(5)
memory usage: 17.6+ MB


In [12]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189021 entries, 0 to 256135
Data columns (total 9 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Perlindungan Isi Rumah           189021 non-null  object 
 1   Alarm Terpasang                  189021 non-null  object 
 2   Risiko Banjir                    189021 non-null  object 
 3   Keanggotaan Keamanan Lingkungan  189021 non-null  object 
 4   Jenis Kepemilikan Properti       189021 non-null  float64
 5   Tipe Properti                    189021 non-null  float64
 6   Risiko Penurunan Tanah           189021 non-null  object 
 7   Tahun Dibangun                   189021 non-null  float64
 8   Premi Asuransi                   189021 non-null  float64
dtypes: float64(4), object(5)
memory usage: 14.4+ MB


In [13]:
# Konversi ke Rupiah (pakai USD)
kurs_usd_idr = 15000
df['Premi Asuransi'] = df['Premi Asuransi'] * kurs_usd_idr
df

C:\Users\T480\AppData\Local\Temp\ipykernel_4752\885554043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Premi Asuransi'] = df['Premi Asuransi'] * kurs_usd_idr


,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi
0,Y,N,Y,N,1.0,1.0,N,1960.0,4122150.0
1,Y,N,Y,N,2.0,2.0,N,1960.0,4632450.0
2,N,Y,Y,Y,1.0,5.0,N,1946.0,789750.0
3,N,N,Y,N,4.0,4.0,N,1870.0,813450.0
4,Y,Y,Y,N,1.0,3.0,N,1960.0,3668700.0
...,...,...,...,...,...,...,...,...,...
256131,Y,N,Y,N,1.0,2.0,N,1980.0,3526200.0
256132,Y,N,Y,N,1.0,3.0,N,1980.0,2910300.0
256133,Y,Y,Y,N,1.0,4.0,N,1900.0,4309500.0
256134,Y,N,Y,N,2.0,4.0,N,1900.0,6863550.0


In [14]:
# Encoding contoh kolom kategorikal
le = LabelEncoder()
for col in ['Perlindungan Isi Rumah', 'Alarm Terpasang', 'Risiko Banjir',
            'Keanggotaan Keamanan Lingkungan', 'Risiko Penurunan Tanah']:
    df[col] = le.fit_transform(df[col])

C:\Users\T480\AppData\Local\Temp\ipykernel_4752\2418017865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
C:\Users\T480\AppData\Local\Temp\ipykernel_4752\2418017865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
C:\Users\T480\AppData\Local\Temp\ipykernel_4752\2418017865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [15]:
df

,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi
0,1,0,1,0,1.0,1.0,0,1960.0,4122150.0
1,1,0,1,0,2.0,2.0,0,1960.0,4632450.0
2,0,1,1,1,1.0,5.0,0,1946.0,789750.0
3,0,0,1,0,4.0,4.0,0,1870.0,813450.0
4,1,1,1,0,1.0,3.0,0,1960.0,3668700.0
...,...,...,...,...,...,...,...,...,...
256131,1,0,1,0,1.0,2.0,0,1980.0,3526200.0
256132,1,0,1,0,1.0,3.0,0,1980.0,2910300.0
256133,1,1,1,0,1.0,4.0,0,1900.0,4309500.0
256134,1,0,1,0,2.0,4.0,0,1900.0,6863550.0


In [16]:
def buat_label_risiko(premi):
    if premi < 2000000:
        return 0  # risiko rendah
    elif premi < 4000000:
        return 1  # risiko sedang
    else:
        return 2  # risiko tinggi

df['Risiko'] = df['Premi Asuransi'].apply(buat_label_risiko)
df

C:\Users\T480\AppData\Local\Temp\ipykernel_4752\1686630672.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Risiko'] = df['Premi Asuransi'].apply(buat_label_risiko)


,Perlindungan Isi Rumah,Alarm Terpasang,Risiko Banjir,Keanggotaan Keamanan Lingkungan,Jenis Kepemilikan Properti,Tipe Properti,Risiko Penurunan Tanah,Tahun Dibangun,Premi Asuransi,Risiko
0,1,0,1,0,1.0,1.0,0,1960.0,4122150.0,2
1,1,0,1,0,2.0,2.0,0,1960.0,4632450.0,2
2,0,1,1,1,1.0,5.0,0,1946.0,789750.0,0
3,0,0,1,0,4.0,4.0,0,1870.0,813450.0,0
4,1,1,1,0,1.0,3.0,0,1960.0,3668700.0,1
...,...,...,...,...,...,...,...,...,...,...
256131,1,0,1,0,1.0,2.0,0,1980.0,3526200.0,1
256132,1,0,1,0,1.0,3.0,0,1980.0,2910300.0,1
256133,1,1,1,0,1.0,4.0,0,1900.0,4309500.0,2
256134,1,0,1,0,2.0,4.0,0,1900.0,6863550.0,2


In [17]:
# Cek distribusi label risiko
print(df['Risiko'].value_counts(normalize=True))

Risiko
1    0.549002
0    0.286132
2    0.164865
Name: proportion, dtype: float64


# Pisahkan Fitur dan Target

In [18]:
X = df.drop(['Premi Asuransi', 'Risiko'], axis=1).values
y_reg = df['Premi Asuransi'].values
y_clf = df['Risiko'].values

In [19]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y_reg = StandardScaler()
y_reg_scaled = scaler_y_reg.fit_transform(y_reg.reshape(-1,1))

In [20]:
X_train, X_test, y_train_reg, y_test_reg, y_train_clf, y_test_clf = train_test_split(
    X_scaled, y_reg_scaled, y_clf, test_size=0.2, random_state=42
)

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras_tuner as kt

def build_model(hp):
    inputs = layers.Input(shape=(X_train.shape[1],))
    x = layers.Dense(units=hp.Int('units', 64, 128, step=32), activation='relu')(inputs)
    x = layers.Dropout(rate=hp.Float('dropout', 0.2, 0.4, step=0.1))(x)
    
    risk_output = layers.Dense(3, activation='softmax', name='risk_output')(x)
    premium_output = layers.Dense(1, activation='linear', name='premium_output')(x)

    model = models.Model(inputs=inputs, outputs=[risk_output, premium_output])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-4, 1e-3, sampling='log')
        ),
        loss={
            'risk_output': 'sparse_categorical_crossentropy',
            'premium_output': 'mse'
        },
        metrics={
            'risk_output': 'accuracy',
            'premium_output': 'mae'
        }
    )
    return model


In [23]:
from keras_tuner import Objective

tuner = kt.RandomSearch(
    build_model,
    objective=Objective("val_risk_output_accuracy", direction="max"),  # ← tambahkan direction
    max_trials=10,
    executions_per_trial=1,
    directory='asuransi_tuning',
    project_name='multioutput_model'
)

In [25]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Subset untuk tuning (lebih cepat)
X_train_sample = X_train[:5000]
y_train_clf_sample = y_train_clf[:5000]
y_train_reg_sample = y_train_reg[:5000]
X_val_sample = X_test[:1000]
y_val_clf_sample = y_test_clf[:1000]
y_val_reg_sample = y_test_reg[:1000]

tuner.search(
    X_train_sample,
    {'risk_output': y_train_clf_sample, 'premium_output': y_train_reg_sample},
    validation_data=(X_val_sample, {'risk_output': y_val_clf_sample, 'premium_output': y_val_reg_sample}),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 00m 08s]
val_risk_output_accuracy: 0.75

Best val_risk_output_accuracy So Far: 0.75
Total elapsed time: 00h 02m 10s


In [26]:
# Ambil model terbaik
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluasi
results = best_model.evaluate(
    X_test,
    {'risk_output': y_test_clf, 'premium_output': y_test_reg},
    batch_size=128,
    verbose=1
)

for name, value in zip(best_model.metrics_names, results):
    if 'risk_output' in name:
        print(f"[Klasifikasi Risiko] {name}: {value:.4f}")
    elif 'premium_output' in name:
        print(f"[Prediksi Premi]     {name}: {value:.4f}")
    else:
        print(f"{name}: {value:.4f}")


c:\Users\T480\Documents\CAPSTONE\fixenv\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1891 - premium_output_loss: 0.5301 - premium_output_mae: 0.5111 - risk_output_accuracy: 0.7386 - risk_output_loss: 0.6590
loss: 1.2015
compile_metrics: 0.6659
[Klasifikasi Risiko] risk_output_loss: 0.5360
[Prediksi Premi]     premium_output_loss: 0.5119


Kesimpulan:

- Model cukup berhasil melakukan klasifikasi risiko dan prediksi premi.
- Akurasi klasifikasi 73.86% → layak, tapi bisa ditingkatkan (misal tuning layer, balancing class, dll).
- MAE 0.5111 (normalized) → cukup kecil, menandakan model tidak terlalu jauh meleset dalam prediksi premi.

In [27]:
best_model.save("ModelPremiProperti.h5")

In [29]:
import joblib

# Simpan scaler fitur
joblib.dump(scaler_X, "scaler_Properti_X.pkl")

# Simpan scaler target regresi
joblib.dump(scaler_y_reg, "scaler_properti_Y.pkl")

['scaler_properti_Y.pkl']